# Práctica 1 

----

## Instrucciones.

#### Los objetivos para esta práctica son los siguientes:

1. Que puedas trabajar en colaboración a través de gitlab y bastión.
2. Que puedas correr un notebook dentro de un issue y rama asignados correctamente.
3. Que puedas limpiar la base de datos.
4. Que generes información a partir de la base de datos.

#### 1. Para correr el presente notebook debes de tener:

Recuerda que para este punto debes de tener tu [set up](https://gitlab.com/egobiernoytp/ccd/knowledge/-/blob/develop/tutorials/kit/set_up.md) completado. 

1. El repositorio de knowledge clonado dentro de bastión con la rama de develop:

`git clone --branch develop https://gitlab.com/egobiernoytp/ccd/knowledge.git`

2. Trabajar dentro de un pyenv exclusivo para `knowledge` y dentro del repositorio de tu bastión:

[Configuración pyenvs](https://gitlab.com/egobiernoytp/ccd/knowledge/-/blob/develop/tutorials/kit/virtual_environments.md)

3. Tener instalados los paquetes de `requirements.txt`, dentro de tu pyenv de knowledge y dentro de bastión:

`pip install -r requirements.txt`

4. Abrir un notebook con el kernel de knowledge.

5. Establecer correctamente tus credenciales para que puedas correr un notebook sin problemas (pueden ser las credenciales de la base de noms o de red_ciudades que deben de estar bien en `config/local/credentials.yaml`).

#### 2. Una vez que tengas esto establecido deberás trabajar con el flujo de trabajo dentro de gitlab y bastión, es decir: 

- Crea un issue en gitlab con el nombre de Practica 1 <tu_nombre>.
- Con el número de issue que te asigna gitlab, generarás una rama en bastión a partir de develop, que tenga el número de issue que te asignó gitlab, y tu nombre. En otras palabras, generarás en bastión una rama con el siguiente nombre: issue_numero_tu_nombre, ejemplo: issue_7_elena. 
- Trabajar tu práctica dentro de tu rama y hacer commits de los cambios en tu trabajo. 
    
#### 3. Una vez que tengas todo establecido podrás correr el presente notebook y hacer la práctica que consiste en lo siguiente:

- Cambia el nombre del presente archivo por práctica_1_tunombre, ejemplo, practica_1_elena.ipynb y haz un primer commit con este cambio. 
- Después, obtener la base de datos del query que se encuentra en este notebook. En ésta, tienes solo dos columnas, una de `etiquetas` y otra de `cantidad`. Como podrás observar la variable de etiquetas tiene tres tipos de información contenida en una sola columna. Tu tarea es separar la información de la columna `etiquetas` por: `usuarios | cantidad | grupos | fechas`. 
- En el notebook de `capacitacion_1` coloqué toda la información necesaria para hacer esta transformación, el truco está en los índices, en las listas y arreglos. 
- Recuerda hacer commits constantes para que se haga un track de tu trabajo. 

#### 4. Una vez limpia la base de datos, esperaría tener la siguiente información:

1. ¿Cuántas variables y observaciones tenemos?
2. Cambiar el tipo de columna correspondiente. Es decir, cambiar a variable str, int o date, dependiendo de la información. 
4. Cantidad de observaciones por grupos. 
5. Obtener dos columnas adicioniales: el día del mes y el día de la semana (lunes, martes, etc). *(Busca en internet como puedes obtener esta información)*. 
6. Describir la variable numérica.

Se realizará la evaluación sobre la rama creada en gitlab, REALIZA COMMITS CONSTANTES. 

¡SUERTE!

E.

:P

In [1]:
import numpy as np
import pandas as pd
import yaml
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils.general import get_db_conn, get_s3_credentials

In [2]:
conn = get_db_conn("../config/local/credentials.yaml")

**Para ver 200 filas en el notebook**

In [3]:
pd.set_option('display.max_rows', 200)

**Para ver 10 columnas en el notebook**

In [4]:
pd.set_option('display.max_columns', 10)

#### Query para obtener la base de datos

In [5]:
 # Query
q = """
    select 
        *
    from 
        raw.base_tarea;
    
"""

In [8]:
df = pd.read_sql(q, conn)

/tmp/ipykernel_246940/1087245975.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)


In [7]:
df.head(15)

,etiquetas,cantidad
0,grupo_1,25264
1,18/2/2022,300
2,U339-OQT2D,50
3,U960-CWLVS,250
4,22/2/2022,350
5,U060-VFRUD,50
6,U576-6HBNT,50
7,U623-8JNNF,50
8,U762-PEBB9,100
9,U960-CWLVS,100


## Limpieza de base

In [9]:
grupos = df[df['etiquetas'].str.contains('_')] #grupos con cantidades totales

In [10]:
grupos = grupos.reset_index()
grupos.head(15)

,index,etiquetas,cantidad
0,0,grupo_1,25264
1,370,grupo_2,21900
2,622,grupo_3,29750
3,916,grupo_4,16050
4,1145,grupo_5,23950
5,1444,grupo_6,11700
6,1603,grupo_7,26650
7,1860,grupo_8,23800
8,2172,grupo_9,32550
9,2613,grupo_10,15200


In [11]:
fechas = df[df['etiquetas'].str.contains('/')]  # fechas con cantidades totales

In [12]:
fechas = fechas.reset_index()
fechas.head(15)

,index,etiquetas,cantidad
0,1,18/2/2022,300
1,4,22/2/2022,350
2,10,23/2/2022,550
3,21,28/2/2022,250
4,24,1/3/2022,812
5,35,2/3/2022,550
6,45,3/3/2022,200
7,47,8/3/2022,812
8,57,9/3/2022,600
9,68,15/3/2022,784


In [14]:
# Para hacer la limpieza
# Generamos un data frame vacío
df2 = pd.DataFrame(columns=['usuarios', 'cantidad','grupos','fechas'])
#df2

In [15]:
# Ubicamos y asignamos en una nueva columna de usuarios y cantidades
df2["usuarios"] = df[df['etiquetas'].str.contains("-")]["etiquetas"]
df2["cantidad"] = df[df['etiquetas'].str.contains("-")]["cantidad"]

In [16]:
df2.head()

,usuarios,cantidad,grupos,fechas
2,U339-OQT2D,50,NaN,NaN
3,U960-CWLVS,250,NaN,NaN
5,U060-VFRUD,50,NaN,NaN
6,U576-6HBNT,50,NaN,NaN
7,U623-8JNNF,50,NaN,NaN


In [17]:
# Creamos dos tuplas vacías
grupo_i = ''
fecha_i = ''

In [18]:
df.loc[1,'etiquetas']

'18/2/2022'

In [20]:
len(df)

13076

In [21]:
 # Loop para ir por cada uno de los renglones y ver si se cumple solo una de las siguientes condiciones.
for i in range(0,len(df)):
    
    if '_' in df.loc[i,'etiquetas']:
        
        grupo_i = df.loc[i,'etiquetas']
        

    elif '/' in df.loc[i,'etiquetas']:
        
        fecha_i = df.loc[i,'etiquetas'] 
        
    else:
        df2.loc[i,'grupos'] = grupo_i
        df2.loc[i,'fechas'] = fecha_i
        
# NOTA: Con if, elif y else solo se ejecutará la primera condición que se evalúe como TRUE

In [22]:
df2.head(5)

,usuarios,cantidad,grupos,fechas
2,U339-OQT2D,50,grupo_1,18/2/2022
3,U960-CWLVS,250,grupo_1,18/2/2022
5,U060-VFRUD,50,grupo_1,22/2/2022
6,U576-6HBNT,50,grupo_1,22/2/2022
7,U623-8JNNF,50,grupo_1,22/2/2022


In [23]:
 df2['cantidad'] = df2['cantidad'].astype('int')df2.reset_index()

,index,usuarios,cantidad,grupos,fechas
0,2,U339-OQT2D,50,grupo_1,18/2/2022
1,3,U960-CWLVS,250,grupo_1,18/2/2022
2,5,U060-VFRUD,50,grupo_1,22/2/2022
3,6,U576-6HBNT,50,grupo_1,22/2/2022
4,7,U623-8JNNF,50,grupo_1,22/2/2022
...,...,...,...,...,...
12097,13071,U676-0X72P,100,grupo_23,31/5/2022
12098,13072,U741-47NHW,50,grupo_23,31/5/2022
12099,13073,U862-JQ5G4,200,grupo_23,31/5/2022
12100,13074,U887-VMC05,50,grupo_23,31/5/2022


In [25]:
df2['cantidad'] = df2['cantidad'].astype('int')

In [26]:
grupos['cantidad'] = grupos['cantidad'].astype('int')

In [28]:
# Comprobamos haber limpieado correctamente la base

In [27]:
df2.groupby("grupos")["cantidad"].sum().sort_values()

grupos
grupo_11      5100
grupo_12      8270
grupo_6      11700
grupo_13     14050
grupo_10     15200
grupo_4      16050
grupo_22     17950
grupo_18     18050
grupo_2      21900
grupo_8      23800
grupo_5      23950
grupo_17     25200
grupo_1      25264
grupo_20     25350
grupo_7      26650
grupo_3      29750
grupo_15     31300
grupo_9      32550
grupo_23     33350
grupo_19     40000
grupo_21     40400
grupo_14     73400
grupo_16    920730
Name: cantidad, dtype: int64

In [29]:
grupos[(['etiquetas','cantidad'])].sort_values('cantidad')

,etiquetas,cantidad
10,grupo_11,5100
11,grupo_12,8270
5,grupo_6,11700
12,grupo_13,14050
9,grupo_10,15200
3,grupo_4,16050
21,grupo_22,17950
17,grupo_18,18050
1,grupo_2,21900
7,grupo_8,23800


## Descripción de datos

#### 1. ¿Cuántas varialbes y observaciones tenemos?

In [30]:
print('Observaciones:',df2.shape[0])

Observaciones: 12102


In [31]:
print('Columnas:',df2.shape[1])

Columnas: 4


#### 2. Cambiar el tipo de columna correspondiente.

In [40]:
df2['cantidad'] = df2['cantidad'].astype('int')
df2['fechas'] = pd.to_datetime(df2['fechas']) 

In [33]:
df2.dtypes

usuarios            object
cantidad             int64
grupos              object
fechas      datetime64[ns]
dtype: object

#### 3. Cantidad de observaciones por grupo

In [35]:
df2.groupby("grupos")["usuarios"].count()

grupos
grupo_1      325
grupo_10     100
grupo_11      72
grupo_12      89
grupo_13     138
grupo_14     170
grupo_15     104
grupo_16    7425
grupo_17     223
grupo_18     165
grupo_19     330
grupo_2      216
grupo_20     295
grupo_21     356
grupo_22     207
grupo_23     216
grupo_3      234
grupo_4      191
grupo_5      254
grupo_6      122
grupo_7      221
grupo_8      270
grupo_9      379
Name: usuarios, dtype: int64

#### 4. Obtener dos columnas adicionales: el día del mes y día de la semana

In [36]:
df2['dia_semana'] = df2['fechas'].dt.day_name()
df2['dia_mes'] = df2['fechas'].dt.day

In [37]:
df2.head()

,usuarios,cantidad,grupos,fechas,dia_semana,dia_mes
2,U339-OQT2D,50,grupo_1,2022-02-18,Friday,18
3,U960-CWLVS,250,grupo_1,2022-02-18,Friday,18
5,U060-VFRUD,50,grupo_1,2022-02-22,Tuesday,22
6,U576-6HBNT,50,grupo_1,2022-02-22,Tuesday,22
7,U623-8JNNF,50,grupo_1,2022-02-22,Tuesday,22


#### 5. Describir variable numérica

In [39]:
df2.describe()

,cantidad,dia_mes
count,12102.000000,12102.000000
mean,122.290861,15.765411
std,279.601130,9.872164
min,50.000000,3.000000
25%,50.000000,5.000000
50%,50.000000,17.000000
75%,100.000000,24.000000
max,7000.000000,31.000000
